In [1]:
import pandas as pd
import json
from scipy import stats
import numpy as np

## 제품 클릭 데이터

In [2]:
read_path = '../csv/click_1911.csv'
click = pd.read_csv(read_path)

click = click.rename(columns={'target_id': 'pid'})

click['pid'] = click['pid'].astype(str)

click.tail()

,updated,pid,keyword
71493417,2019-11-07 23:59:55,95136963,NaN
71493418,2019-11-07 23:59:55,96203304,NaN
71493419,2019-11-07 23:59:55,106010080,오소이
71493420,2019-11-07 23:59:55,111370984,멜론
71493421,2019-11-07 23:59:55,111111572,NaN


## 제품 태그 데이터

In [3]:
read_path = '../csv/product_keyword_1911.csv'
product = pd.read_csv(read_path)

# click = click.rename(columns={'target_id': 'pid'})

product['pid'] = product['pid'].astype(str)

product.tail()

,pid,create_date,keyword
427470,111374143,2019-11-07 23:55:44.494597,"나이키패딩XL,스우시패딩,양면패딩"
427471,111374207,2019-11-07 23:56:31.566875,NaN
427472,111374271,2019-11-07 23:57:21.458948,"아이폰,갤럭시,갤럭시s10,노트10플러스,갤럭시노트"
427473,111374335,2019-11-07 23:58:14.403435,나이키 후드점퍼
427474,111374399,2019-11-07 23:59:22.621879,"소아온,나와호랑이님,라이트노벨,라노벨"


## merge click with product data

In [5]:
click_product = pd.merge(click, product, on='pid', suffixes=('_click', '_product'))

click_product

,updated,pid,keyword_click,create_date,keyword_product
0,2019-11-01 00:01:06,110946993,NaN,2019-11-01 00:00:24.533452,"셀린자켓,by1031"
1,2019-11-01 00:05:49,110946993,NaN,2019-11-01 00:00:24.533452,"셀린자켓,by1031"
2,2019-11-01 00:01:10,110946993,NaN,2019-11-01 00:00:24.533452,"셀린자켓,by1031"
3,2019-11-01 00:02:35,110946993,NaN,2019-11-01 00:00:24.533452,"셀린자켓,by1031"
4,2019-11-01 00:16:46,110946993,NaN,2019-11-01 00:00:24.533452,"셀린자켓,by1031"
...,...,...,...,...,...
17632037,2019-11-07 23:59:37,111373252,공기계 울산,2019-11-07 23:43:37.648117,"Q7,Q720,중고폰,부산중고폰,울산중고폰"
17632038,2019-11-07 23:59:37,111374187,NaN,2019-11-07 23:56:14.573294,"넷플릭스,#영화,#TV"
17632039,2019-11-07 23:59:54,111374426,NaN,2019-11-07 23:59:46.963877,"디즈니,디즈니공주인형,공주인형,엘레나공주,디즈니인형"
17632040,2019-11-07 23:59:54,111374346,NaN,2019-11-07 23:58:30.799362,NaN


In [7]:
click_product_keyword = click_product.dropna(axis=0)

click_product_keyword

,updated,pid,keyword_click,create_date,keyword_product
87,2019-11-07 23:49:07,110946993,숏코트,2019-11-01 00:00:24.533452,"셀린자켓,by1031"
304,2019-11-01 00:24:14,110946997,베이지 숏패딩,2019-11-01 00:00:26.454526,"100,105"
310,2019-11-01 01:58:38,110946997,베이지 패딩,2019-11-01 00:00:26.454526,"100,105"
312,2019-11-01 03:20:30,110946997,숏패딩 베이지,2019-11-01 00:00:26.454526,"100,105"
313,2019-11-01 07:13:19,110946997,숏패딩,2019-11-01 00:00:26.454526,"100,105"
...,...,...,...,...,...
17631995,2019-11-07 23:56:47,111371167,아이폰11,2019-11-07 23:14:50.332713,"아이폰8,중고폰,가개통,A급,아이폰11"
17631997,2019-11-07 23:56:52,111371447,장식품,2019-11-07 23:18:36.557912,"골동품,수집품,장식품,희귀품,은"
17632011,2019-11-07 23:57:49,111331826,등산자켓,2019-11-07 13:02:11.602752,"에이글,등산복,방수자켓,등산자켓"
17632034,2019-11-07 23:59:23,111363377,구스,2019-11-07 21:29:29.043651,"패딩,구스패딩,구스,잠바,점퍼"


In [24]:

def matching_keyword(row):
    
    result = 0
    
    for each_text in row['keyword_click']:
    
        if each_text in row['keyword_product']:
            result += 1
            
    if result > 0:
        return 1
    else:
        return None

click_product_keyword['click_by_keyword'] = click_product_keyword.apply(matching_keyword, axis=1)

click_product_keyword

/Users/quicket/python-bunjang/venv/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,updated,pid,keyword_click,create_date,keyword_product,click_by_keyword
87,2019-11-07 23:49:07,110946993,숏코트,2019-11-01 00:00:24.533452,"셀린자켓,by1031",NaN
304,2019-11-01 00:24:14,110946997,베이지 숏패딩,2019-11-01 00:00:26.454526,"100,105",NaN
310,2019-11-01 01:58:38,110946997,베이지 패딩,2019-11-01 00:00:26.454526,"100,105",NaN
312,2019-11-01 03:20:30,110946997,숏패딩 베이지,2019-11-01 00:00:26.454526,"100,105",NaN
313,2019-11-01 07:13:19,110946997,숏패딩,2019-11-01 00:00:26.454526,"100,105",NaN
...,...,...,...,...,...,...
17631995,2019-11-07 23:56:47,111371167,아이폰11,2019-11-07 23:14:50.332713,"아이폰8,중고폰,가개통,A급,아이폰11",1.0
17631997,2019-11-07 23:56:52,111371447,장식품,2019-11-07 23:18:36.557912,"골동품,수집품,장식품,희귀품,은",1.0
17632011,2019-11-07 23:57:49,111331826,등산자켓,2019-11-07 13:02:11.602752,"에이글,등산복,방수자켓,등산자켓",1.0
17632034,2019-11-07 23:59:23,111363377,구스,2019-11-07 21:29:29.043651,"패딩,구스패딩,구스,잠바,점퍼",1.0


In [25]:
click_by_pid = click_product_keyword.groupby(['pid'], as_index=False).agg({'click_by_keyword' : 'count'})

click_by_pid

,pid,click_by_keyword
0,110946976,11
1,110946977,3
2,110946980,13
3,110946981,4
4,110946983,133
...,...,...
262622,111374234,1
262623,111374247,0
262624,111374267,0
262625,111374268,1


In [26]:
click_by_pid_keyword = click_by_pid[click_by_pid['click_by_keyword'] > 0 ]

click_by_pid_keyword

,pid,click_by_keyword
0,110946976,11
1,110946977,3
2,110946980,13
3,110946981,4
4,110946983,133
...,...,...
262619,111374145,1
262621,111374225,1
262622,111374234,1
262625,111374268,1


In [27]:

def matching_full_keyword(row):
            
    if row['keyword_click'] in row['keyword_product']:
        return 1
    else:
        return None

click_product_keyword['click_by_full_keyword'] = click_product_keyword.apply(matching_full_keyword, axis=1)

click_product_keyword

/Users/quicket/python-bunjang/venv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,updated,pid,keyword_click,create_date,keyword_product,click_by_keyword,click_by_full_keyword
87,2019-11-07 23:49:07,110946993,숏코트,2019-11-01 00:00:24.533452,"셀린자켓,by1031",NaN,NaN
304,2019-11-01 00:24:14,110946997,베이지 숏패딩,2019-11-01 00:00:26.454526,"100,105",NaN,NaN
310,2019-11-01 01:58:38,110946997,베이지 패딩,2019-11-01 00:00:26.454526,"100,105",NaN,NaN
312,2019-11-01 03:20:30,110946997,숏패딩 베이지,2019-11-01 00:00:26.454526,"100,105",NaN,NaN
313,2019-11-01 07:13:19,110946997,숏패딩,2019-11-01 00:00:26.454526,"100,105",NaN,NaN
...,...,...,...,...,...,...,...
17631995,2019-11-07 23:56:47,111371167,아이폰11,2019-11-07 23:14:50.332713,"아이폰8,중고폰,가개통,A급,아이폰11",1.0,1.0
17631997,2019-11-07 23:56:52,111371447,장식품,2019-11-07 23:18:36.557912,"골동품,수집품,장식품,희귀품,은",1.0,1.0
17632011,2019-11-07 23:57:49,111331826,등산자켓,2019-11-07 13:02:11.602752,"에이글,등산복,방수자켓,등산자켓",1.0,1.0
17632034,2019-11-07 23:59:23,111363377,구스,2019-11-07 21:29:29.043651,"패딩,구스패딩,구스,잠바,점퍼",1.0,1.0


In [28]:
click_by_pid = click_product_keyword.groupby(['pid'], as_index=False).agg({'click_by_full_keyword' : 'count'})

click_by_pid

,pid,click_by_full_keyword
0,110946976,0
1,110946977,3
2,110946980,8
3,110946981,4
4,110946983,106
...,...,...
262622,111374234,1
262623,111374247,0
262624,111374267,0
262625,111374268,1


In [29]:
click_by_pid_keyword = click_by_pid[click_by_pid['click_by_full_keyword'] > 0 ]

click_by_pid_keyword

,pid,click_by_full_keyword
1,110946977,3
2,110946980,8
3,110946981,4
4,110946983,106
5,110946984,1
...,...,...
262619,111374145,1
262621,111374225,1
262622,111374234,1
262625,111374268,1


In [23]:
click_product_keyword.drop(['click_by_keyword'], axis=1, inplace=True)

click_product_keyword

,updated,pid,keyword_click,create_date,keyword_product
87,2019-11-07 23:49:07,110946993,숏코트,2019-11-01 00:00:24.533452,"셀린자켓,by1031"
304,2019-11-01 00:24:14,110946997,베이지 숏패딩,2019-11-01 00:00:26.454526,"100,105"
310,2019-11-01 01:58:38,110946997,베이지 패딩,2019-11-01 00:00:26.454526,"100,105"
312,2019-11-01 03:20:30,110946997,숏패딩 베이지,2019-11-01 00:00:26.454526,"100,105"
313,2019-11-01 07:13:19,110946997,숏패딩,2019-11-01 00:00:26.454526,"100,105"
...,...,...,...,...,...
17631995,2019-11-07 23:56:47,111371167,아이폰11,2019-11-07 23:14:50.332713,"아이폰8,중고폰,가개통,A급,아이폰11"
17631997,2019-11-07 23:56:52,111371447,장식품,2019-11-07 23:18:36.557912,"골동품,수집품,장식품,희귀품,은"
17632011,2019-11-07 23:57:49,111331826,등산자켓,2019-11-07 13:02:11.602752,"에이글,등산복,방수자켓,등산자켓"
17632034,2019-11-07 23:59:23,111363377,구스,2019-11-07 21:29:29.043651,"패딩,구스패딩,구스,잠바,점퍼"
